In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, f1_score
from sklearn import metrics
import seaborn as sns
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, f1_score, confusion_matrix
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

In [207]:
sessions_raw=pd.read_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/sessions.csv")
train_users_2_raw=pd.read_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/train_users_2.csv")
test_users_raw=pd.read_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/test_users.csv")
countries_raw=pd.read_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/countries.csv")
age_gender_bkts_raw=pd.read_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/age_gender_bkts.csv")

In [208]:
def func_get_dummy_variables_for_categorical_variables(dataset, variable_name):
    
    dummy_variables_series=pd.get_dummies(dataset[variable_name])
    dummy_variables_series_df=pd.DataFrame(dummy_variables_series)
    
    for column in dummy_variables_series_df.columns:
        new_column_name=variable_name+'_'+str(column)
        if new_column_name in dataset.columns:
            del dataset[new_column_name]
        
        dummy_variables_series_df=dummy_variables_series_df.rename(columns={column: new_column_name})
        
        
    del dataset[variable_name]
    #del dummy_variables_series_df[dummy_variables_series_df.columns[-1]]
    return dummy_variables_series_df

In [209]:
# Create a new dataframe to include the activity count by user:
user_activity_count=sessions_raw.groupby('user_id').size()
user_activity_count=user_activity_count.reset_index()
user_activity_count=user_activity_count.rename(columns={0: 'session count'})
user_activity_count.sort_values(by='session count', ascending=False, inplace=True)

# Create a new dataframe to include the total time spent on Airbnb by user:
user_total_time_spent=sessions_raw.groupby('user_id')['secs_elapsed'].sum()
user_total_time_spent=user_total_time_spent.reset_index()
user_total_time_spent=user_total_time_spent.rename(columns={'secs_elapsed': 'Total time spent (in seconds)'})
user_total_time_spent.sort_values(by='Total time spent (in seconds)', ascending=False, inplace=True)


In [220]:
# Clean up all the date and timestamp fields from the test_users_raw data frame:
test_users_raw['date_account_created']=pd.to_datetime(test_users_raw['date_account_created'])
test_users_raw['date_first_booking']=pd.to_datetime(test_users_raw['date_first_booking'])
test_users_raw['timestamp_first_active_cleaned']=pd.to_datetime(test_users_raw['timestamp_first_active'].apply(lambda x: str(x)[:8]).apply(lambda x: str(x)[:4]) +'-'+ test_users_raw['timestamp_first_active'].apply(lambda x: str(x)[:8]).apply(lambda x: str(x)[4:6]) + '-' + test_users_raw['timestamp_first_active'].apply(lambda x: str(x)[:8]).apply(lambda x: str(x)[6:]))


#Make sure the user training dataset is at the user id level
user_testing_dataset_check_duplicate_df=test_users_raw.groupby('id').size().reset_index()

#Create new variable number_of_days_account_creation: number of days since the account has been created (number of days between date_account_created and date_first_booking)
test_users_raw['number_of_days_account_creation']=test_users_raw['date_first_booking']-test_users_raw['date_account_created']

#Create new variable number_of_active_days: number of days since the user has been active (number of days between timestamp_first_active_cleaned and date_first_booking )
test_users_raw['number_of_active_days']=test_users_raw['date_first_booking']-test_users_raw['timestamp_first_active_cleaned']


#Clean up the age
test_users_raw['age_computed']=test_users_raw['age']
test_users_raw.loc[test_users_raw['age_computed']>=122,'age_computed']=122
test_users_raw.loc[test_users_raw['age_computed']<18,'age_computed']=18


#Create some other additional fields:
test_users_raw['number_of_days_account_creation_numeric_value']=test_users_raw['number_of_days_account_creation'].dt.days
test_users_raw['number_of_active_days_numeric_value']=test_users_raw['number_of_active_days'].dt.days

test_users_raw.loc[test_users_raw['number_of_days_account_creation_numeric_value']==0,'Account_creation_before_booking_flag']=1
test_users_raw.loc[test_users_raw['number_of_days_account_creation_numeric_value']>0,'Account_creation_before_booking_flag']=2
test_users_raw.loc[test_users_raw['number_of_days_account_creation_numeric_value']<0,'Account_creation_before_booking_flag']=0


In [221]:

#Merge the user training data and user activiy data together:
user_final_testing_dataset=pd.merge(left=test_users_raw, right=user_activity_count, left_on="id", right_on="user_id", how='left')
user_final_testing_dataset=pd.merge(left=user_final_testing_dataset, right=user_total_time_spent, left_on="id", right_on="user_id", how='left')

#Adding the activity count by action type
sessions_raw_action=sessions_raw[['user_id', 'action']]
counted=sessions_raw_action.groupby(['user_id','action'])['user_id'].size()
counted_df=pd.DataFrame(counted)
counted_df.columns=['session count']
counted_df.reset_index(inplace=True)
session_count_by_action=counted_df.pivot_table(index='user_id', columns='action', values='session count', aggfunc=np.sum)


In [222]:
for column in session_count_by_action.columns:
    session_count_by_action.rename(columns={column: "Session Count by action name_" + column}, inplace=True)

In [223]:
session_count_by_action=session_count_by_action.reset_index().rename_axis(None, axis=1)
session_count_by_action.set_index('user_id', inplace=True)
session_count_by_action.fillna(0, inplace=True)
#Merge the session count by action with the user training dataset:
user_final_testing_dataset.set_index('id', inplace=True)


user_final_testing_dataset=pd.merge(user_final_testing_dataset, session_count_by_action, left_index=True, right_index=True, how='left')

#Create three different lists of column names for categorical variables, continuous variables and datetime variables
categorical_variables=['signup_flow', 'gender', 'signup_method','language', 'affiliate_channel','affiliate_provider',  'first_affiliate_tracked','signup_app', 'first_device_type', 'first_browser', 'Account_creation_before_booking_flag'  ]
continuous_variables=['age_computed', 'number_of_days_account_creation', 'number_of_active_days_numeric_value', 'number_of_days_account_creation_numeric_value' , 'session count', 'Total time spent (in seconds)']
timestamp_variables=['timestamp_first_active_cleaned', 'date_account_created','date_first_booking' ]


#  only include the variables that will be included in the model:

user_testing_dataset=user_final_testing_dataset
user_testing_dataset['date_account_created']=pd.to_datetime(user_testing_dataset['date_account_created'])
user_testing_dataset['timestamp_first_active_cleaned']=pd.to_datetime(user_testing_dataset['timestamp_first_active_cleaned'])
columns_to_include_listing=[ 'date_account_created', 'timestamp_first_active_cleaned', 'gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked','signup_app', 'first_device_type', 'first_browser','age_computed',   'Account_creation_before_booking_flag', 'Total time spent (in seconds)',  'session count']


for column in [i for i in user_testing_dataset.columns if 'Session Count by action' in i]:
    columns_to_include_listing.append(column)

user_testing_dataset_update=user_testing_dataset[columns_to_include_listing]

user_testing_dataset_update['first_browser']=user_testing_dataset_update['first_browser'].astype("str")
user_testing_dataset_update['gender']=user_testing_dataset_update['gender'].astype("str")

user_testing_dataset_update.loc[user_testing_dataset_update['first_browser']=='-unknown-', 'first_browser']='Unknown'
user_testing_dataset_update.loc[user_testing_dataset_update['gender']=='-unknown-', 'gender']='Unknown'

#Create variables including day of week, day of month, month and year for both account creation date and first active time stamp

user_testing_dataset_update['Account_creation_date_month'] = user_testing_dataset_update['date_account_created'].dt.month
user_testing_dataset_update['Account_creation_date_year'] = user_testing_dataset_update['date_account_created'].dt.year
user_testing_dataset_update['Account_creation_date_day']=user_testing_dataset_update['date_account_created'].dt.day
user_testing_dataset_update['Account_creation_date_day_of_week']=user_testing_dataset_update['date_account_created'].dt.dayofweek

user_testing_dataset_update['first_active_date_month']=user_testing_dataset_update['timestamp_first_active_cleaned'].dt.month
user_testing_dataset_update['first_active_date_day']=user_testing_dataset_update['timestamp_first_active_cleaned'].dt.day
user_testing_dataset_update['first_active_date_year']=user_testing_dataset_update['timestamp_first_active_cleaned'].dt.year
user_testing_dataset_update['first_active_date_dayofweek']=user_testing_dataset_update['timestamp_first_active_cleaned'].dt.dayofweek

C:\Users\yuka.abe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuka.abe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuka.abe\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [224]:
# Convert the categorical variables into categorical variables:
for variable_name in ['gender', 'signup_method', 'signup_flow', 'language','affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked','signup_app', 'first_device_type', 'first_browser', 'Account_creation_before_booking_flag']:
    user_testing_dataset_update[variable_name]=pd.Categorical(user_testing_dataset_update[variable_name])

user_testing_dataset_update['first_affiliate_tracked']=user_testing_dataset_update['first_affiliate_tracked'].astype('str')
user_testing_dataset_update['first_affiliate_tracked'].fillna("Unknown", inplace=True)


C:\Users\yuka.abe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\yuka.abe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\yuka.abe\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [225]:
#Create Age_bucket variable for age:
user_testing_dataset_update.loc[(user_testing_dataset_update['age_computed']>17.999) & (user_testing_dataset_update['age_computed']<=26.0), 'age_bucket' ]='(17.999, 26.0]'
user_testing_dataset_update.loc[(user_testing_dataset_update['age_computed']>26.0) & (user_testing_dataset_update['age_computed']<=28.0), 'age_bucket' ]='(26.0, 28.0]'
user_testing_dataset_update.loc[(user_testing_dataset_update['age_computed']>28.0) & (user_testing_dataset_update['age_computed']<=31.0), 'age_bucket' ]='(28.0, 31.0]'
user_testing_dataset_update.loc[(user_testing_dataset_update['age_computed']>31.0) & (user_testing_dataset_update['age_computed']<=34.0), 'age_bucket' ]='(31.0, 34.0]'
user_testing_dataset_update.loc[(user_testing_dataset_update['age_computed']>34.0) & (user_testing_dataset_update['age_computed']<=37.0), 'age_bucket' ]='(34.0, 37.0]'
user_testing_dataset_update.loc[(user_testing_dataset_update['age_computed']>37.0) & (user_testing_dataset_update['age_computed']<=43.0), 'age_bucket' ]='(37.0, 43.0]'
user_testing_dataset_update.loc[(user_testing_dataset_update['age_computed']>43.0) & (user_testing_dataset_update['age_computed']<=53.0), 'age_bucket' ]='(43.0, 53.0]'
user_testing_dataset_update.loc[(user_testing_dataset_update['age_computed']>53.0) & (user_testing_dataset_update['age_computed']<=122.0), 'age_bucket' ]='(53.0, 122.0]'
user_testing_dataset_update.loc[pd.isnull(user_training_dataset_update['age_computed']),'age_bucket']='Unknown'
user_testing_dataset_update['age_bucket']=pd.Categorical(user_testing_dataset_update['age_bucket'])



In [226]:
#For the total time spent in seconds and total session count variables, fill out the null values with 0.
user_testing_dataset_update['Total time spent (in seconds)_fill_null_zero']=user_testing_dataset_update['Total time spent (in seconds)'].fillna(0 )
user_testing_dataset_update['session count_fill_null_zero']=user_testing_dataset_update['session count'].fillna(0)

user_testing_dataset_update['Total time spent (in seconds)_bucket']=pd.qcut(user_testing_dataset_update['Total time spent (in seconds)'], 5, duplicates='drop')

user_testing_dataset_update['Total time spent (in seconds)_bucket']=user_testing_dataset_update['Total time spent (in seconds)_bucket'].astype('str')
user_testing_dataset_update.loc[user_training_dataset_update['Total time spent (in seconds)_bucket']=='nan','Total time spent (in seconds)_bucket']='Unknown'
user_testing_dataset_update['Total time spent (in seconds)_bucket']=pd.Categorical(user_testing_dataset_update['Total time spent (in seconds)_bucket'])

user_testing_dataset_update['session count_bucket']=pd.qcut(user_testing_dataset_update['session count'], 10, duplicates='drop')
user_testing_dataset_update['session count_bucket']=user_testing_dataset_update['session count_bucket'].astype('str')
user_testing_dataset_update.loc[user_testing_dataset_update['session count_bucket']=='nan','session count_bucket']='Unknown'
user_testing_dataset_update['session count_bucket']=pd.Categorical(user_testing_dataset_update['session count_bucket'])

session_count_by_action_name_column_list=user_testing_dataset_update.columns[user_testing_dataset_update.columns.str.contains('action')]
user_testing_dataset_update[session_count_by_action_name_column_list]=user_testing_dataset_update[session_count_by_action_name_column_list].fillna(0)

# Replace the "-unknown-" with "unknown" for the gender and first_browser fields
user_testing_dataset_update_binary_classfication=user_testing_dataset_update.copy()
user_testing_dataset_update_binary_classfication.loc[user_testing_dataset_update_binary_classfication['first_browser']=='-unknown-', 'first_browser']='Unknown'
user_testing_dataset_update_binary_classfication.loc[user_testing_dataset_update_binary_classfication['gender']=='-unknown-', 'gender']='Unknown'

#For session count and total time spent, create two additional fields to fill the null value for average number
total_time_spent_mean=user_testing_dataset_update_binary_classfication.loc[user_testing_dataset_update_binary_classfication['Total time spent (in seconds)_fill_null_zero']!=0,'Total time spent (in seconds)_fill_null_zero'].mean()
session_count_mean=user_testing_dataset_update_binary_classfication.loc[user_testing_dataset_update_binary_classfication['session count_fill_null_zero']!=0,'session count_fill_null_zero'].mean()
user_testing_dataset_update_binary_classfication['Total time spent (in seconds)_fill_null_mean']=user_testing_dataset_update_binary_classfication['Total time spent (in seconds)_fill_null_zero'].replace(0, total_time_spent_mean)
user_testing_dataset_update_binary_classfication['session count_fill_null_mean']=user_training_dataset_update_binary_classfication['session count_fill_null_zero'].replace(0, session_count_mean)

user_testing_dataset_update_binary_classfication1=user_testing_dataset_update_binary_classfication.copy()
session_count_by_action_name_column_list=user_testing_dataset_update_binary_classfication1.columns[user_testing_dataset_update_binary_classfication1.columns.str.contains('action')]


In [227]:
feature_to_include_list=[
#Categorical Variables:
'gender',
'signup_method',
'signup_flow',
'language',
'affiliate_channel',
'affiliate_provider',
'first_affiliate_tracked',
'signup_app',
'first_device_type',
'first_browser',
#'trip_booking_flag',
'Account_creation_before_booking_flag',

# Buckets created for age, total time spent and session count:
'age_bucket',
'Total time spent (in seconds)_bucket',                    
'session count_bucket',
    
#Continuous Variables:
'Total time spent (in seconds)_fill_null_zero',  
'session count_fill_null_zero',
'Total time spent (in seconds)_fill_null_mean', 
'session count_fill_null_mean'  ,

# Year, month, date, day of the week derived from the account creation date:
'Account_creation_date_year',
'Account_creation_date_month',
'Account_creation_date_day',
'Account_creation_date_day_of_week',
    
  

# Year, month, date, day of the week derived from the first active date:
'first_active_date_year',
'first_active_date_month',
'first_active_date_day',
'first_active_date_dayofweek' ]

In [228]:
#Append all the session count by action type variables to the feature list:

for c in session_count_by_action_name_column_list:
    feature_to_include_list.append(c)

user_testing_dataset_update_binary_classfication1=user_testing_dataset_update_binary_classfication1[feature_to_include_list]         



In [229]:
Categorical_variable_list=['gender',
'signup_method',
'signup_flow',
'language',
'affiliate_channel',
'affiliate_provider',
'first_affiliate_tracked',
'signup_app',
'first_device_type',
'first_browser',
'age_bucket',
'Total time spent (in seconds)_bucket',                   
'session count_bucket',
'Account_creation_date_year',
'Account_creation_date_month', 'Account_creation_date_day',
'Account_creation_date_day_of_week', 'first_active_date_year',
'first_active_date_month', 'first_active_date_day',
'first_active_date_dayofweek',
'Account_creation_before_booking_flag']

In [230]:
for variable_name in Categorical_variable_list:
    df_dummy=func_get_dummy_variables_for_categorical_variables(user_testing_dataset_update_binary_classfication1, variable_name)
    user_testing_dataset_update_binary_classfication1=user_testing_dataset_update_binary_classfication1.merge(df_dummy, on=None, left_index=True, right_index=True)


In [231]:
for column in user_testing_dataset_update_binary_classfication1.columns:
    row_count=user_testing_dataset_update_binary_classfication1[column].count()
    if row_count<62096:
        print(column)

In [232]:
feature_list=list(user_testing_dataset_update_binary_classfication1.columns)
feature_list.remove('Total time spent (in seconds)_fill_null_zero')
feature_list.remove('session count_fill_null_zero')
feature_list.remove('Total time spent (in seconds)_fill_null_mean')
feature_list.remove('session count_fill_null_mean')

feature_list1=list(user_testing_dataset_update_binary_classfication1.columns)                                                                                          
bucketized_feature=list(user_testing_dataset_update_binary_classfication1.columns[user_testing_dataset_update_binary_classfication1.columns.str.contains('bucket')])
age_bucketized_feature=list(user_testing_dataset_update_binary_classfication1.columns[user_testing_dataset_update_binary_classfication1.columns.str.contains('age_bucket')])
for feature in age_bucketized_feature:
    bucketized_feature.remove(feature)

#feature_list1.remove('trip_booking_flag')
for feature in bucketized_feature:
    feature_list1.remove(feature)
feature_list1.remove('Total time spent (in seconds)_fill_null_mean')
feature_list1.remove('session count_fill_null_mean')



In [233]:
X_TEST_binary=user_testing_dataset_update_binary_classfication1[feature_list1]

In [234]:
sample_training_dataset_binary=pd.read_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/user_training_dataset_sample.csv")

In [235]:
feature_list_to_delete=[i for i in X_TEST_binary.columns if i not in sample_training_dataset_binary.columns]

In [236]:
feature_list_to_add=[i for i in sample_training_dataset_binary.columns if i not in X_TEST_binary.columns  ]

# Delete the features that are not included in the model:

In [240]:
for feature in feature_list_to_delete:
    del X_TEST_binary[feature]

# Add the features that are included in the model but not in the testing set:

In [241]:
for feature in feature_list_to_add:
    X_TEST_binary[feature]=0

C:\Users\yuka.abe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Export testing set for the binary classification:

In [245]:
X_TEST_binary.shape

(62096, 629)

In [246]:
X_TEST_binary.to_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/testing_dataset_for_binary_classification.csv")

In [247]:
X_TEST_binary.head()

,Total time spent (in seconds)_fill_null_zero,session count_fill_null_zero,Session Count by action name_10,Session Count by action name_11,Session Count by action name_12,Session Count by action name_15,Session Count by action name_about_us,Session Count by action name_accept_decline,Session Count by action name_account,Session Count by action name_acculynk_bin_check_failed,...,first_active_date_year_2013,first_active_date_month_1,first_active_date_month_2,first_active_date_month_3,first_active_date_month_4,first_active_date_month_5,first_active_date_month_6,first_active_date_month_10,first_active_date_month_11,first_active_date_month_12
id,,,,,,,,,,,,,,,,,,,,,
5uwns89zht,119187.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
jtl0dijy2j,250119.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
xx0ulgorjt,975575.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
6c6puo6ix0,123001.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
czqhjk3yfe,454023.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


# Generate the testing dataset for multi class model:

In [256]:
X_TEST_multiclass=user_testing_dataset_update_binary_classfication1[feature_list1]

In [257]:
sample_training_dataset_multiclass=pd.read_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/user_training_dataset_sample_for_multi_class_classification.csv")

In [258]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X_TEST_multiclass.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_TEST_multiclass.columns.values]

In [259]:
feature_list_to_delete_multiclass=[i for i in X_TEST_multiclass.columns if i not in sample_training_dataset_multiclass.columns]
feature_list_to_add_multiclass=[i for i in sample_training_dataset_multiclass.columns if i not in X_TEST_multiclass.columns  ]

# Delete the features that are not included in the model:

In [262]:
for feature in feature_list_to_delete_multiclass:
    del X_TEST_multiclass[feature]

In [263]:
for feature in feature_list_to_add_multiclass:
    X_TEST_multiclass[feature]=0

C:\Users\yuka.abe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [264]:
X_TEST_multiclass.head()

,Total time spent (in seconds)_fill_null_zero,session count_fill_null_zero,Session Count by action name_10,Session Count by action name_11,Session Count by action name_12,Session Count by action name_15,Session Count by action name_about_us,Session Count by action name_accept_decline,Session Count by action name_account,Session Count by action name_acculynk_bin_check_failed,...,first_active_date_month_5,first_active_date_month_6,first_active_date_month_10,first_active_date_month_11,first_active_date_month_12,first_active_date_year_2009,first_active_date_year_2010,first_active_date_year_2011,first_active_date_year_2012,first_active_date_year_2013
id,,,,,,,,,,,,,,,,,,,,,
5uwns89zht,119187.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
jtl0dijy2j,250119.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
xx0ulgorjt,975575.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
6c6puo6ix0,123001.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
czqhjk3yfe,454023.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [268]:
X_TEST_multiclass.to_csv("C:/Users/yuka.abe/Desktop/Airbnb Kaggle DataSet/testing_dataset_for_multiclass_classification.csv")